In [1]:
#default_export export
#export
from textwrap import TextWrapper
from keyword import iskeyword
from pathlib import Path
import nbformat
import os, re

from typing import List, Tuple, Pattern, Match, Union

# Export a Jupyter Notebook

Convert a Jupyter Notebook into a python module using `exportNotebook` and `exportNotebooks`.

## Quick guide
In order to export cells to a notebook add:
 - `#export` for every cell you want to export
 - `#exporti` for every cell you want to export without adding it to \_\_all\_\_ 
 - `#export[i] module_name` for cells you want to export in a module named **module_name**
 
For giving a name for the python module from within the notebook add `#default_export module_name` on any code cell in the notebook (it is reccomended that you add this at the very beggining for clarity)

## Examples

In [2]:
#default_export demo_name

Since there is already a `#default_export export` in the first cell of the notebook, that name will be used.
<br>

### Note that private functions, classes or variables will not be included in the \_\_all\_\_ of the exported module

In [3]:
#export
CELL_TYPE = 'cell_type'
SOURCE = 'source'
CELLS = 'cells'
CODE = 'code'

In [4]:
#export
MAYBE_MODULE = 0
NO_MODULE = 1
MODULE = 2

In [5]:
nb_name = 'Export.ipynb'
nb_path = Path('./')
nb_path = nb_path/nb_name

In [6]:
#export
def read_notebook(nb_name: str) -> nbformat.notebooknode.NotebookNode:
    """
    Read a Jupyter Notebook using the nbformat module
    
    :param nb_name: The name of the jupyter notebook file
    :return: The notebook as a dictionary
    """
    with open(Path(nb_name),'r', encoding='utf8') as f: 
        return nbformat.reads(f.read(), as_version=4)

In [7]:
test_nb = read_notebook('Export.ipynb')

In [8]:
test_nb.keys()

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])

In [9]:
test_nb['cells'][0]

{'cell_type': 'code',
 'execution_count': 1,
 'metadata': {},
 'outputs': [],
 'source': '#default_export export\n#export\nfrom textwrap import TextWrapper\nfrom keyword import iskeyword\nfrom pathlib import Path\nimport nbformat\nimport os, re\n\nfrom typing import List, Tuple, Pattern, Match, Union'}

In [10]:
#export
def file_exists(file_name: str) -> bool:
    """
    Check if there exists a file with a given filename
    
    :param file_name: The name of the file
    :return: If there is a file with that name at the given path
    """
    return Path(file_name).is_file()

In [11]:
#export
def create_empty_file(module_name:str, nb_path: Path, include_header: bool=True) -> None:
    """
    Create an empty python module with given name relative to the path
    of the notebook that will be exported in that module
    
    :param module_name: The name of the python module
    :param nb_path: The path to the notebook that will be exported
    :param include_header: Flag for writing 'AUTOGENERATED' on the new python module
    """
    
    # get the path where the new file will be created (relative to the notebook)
    file_path = os.path.relpath(nb_path).replace('\\', '/')
    
    # make sure the python module ends with a .py
    if not module_name.endswith('.py'):
        module_name += '.py'
    
    with open(module_name, 'w') as f:
        if include_header:
            f.write(f"# AUTOGENERATED from Notebook: {file_path}!")
        f.write('\n\n__all__ = []')

# Regular Expression

https://docs.python.org/3/howto/regex.html#grouping

# Regular Expression mini-cheatsheet


## Special Chracters
`.` - Matches any character except line terminators like `\n` <br>
`+` - Greedily matches the expression to its left 1 or more times. <br>
`^` - Matches the expression to its right at the start of a string. It matches every such instance before each `\n` in the string. <br>
`$` - Matches the expression to its left at the end of a string. It matches every such instance before each `\n` in the string.<br>

## Character Classes
`\s` - Matches whitespace characters, which include the `\t`, `\n`, `\r`, and space characters. <br>
`\S` - Matches non-whitespace characters. <br>

## Groups
`(?#...)` - A comment. Contents are for us to read, not for matching <br>
`(?PAB)` - Matches the expression AB, and it can be accessed with the group name. <br>
`(?:A)` - Matches the expression as represented by A, but unlike `(?PAB)`, it cannot be retrieved afterwards.<br>

For a more complex cheatsheet checkout [this link](https://www.dataquest.io/blog/regex-cheatsheet/).

In [12]:
#export
def _make_regex(
    body: str, 
    expect_module: int, 
    comment: str, 
    prefix:str =r"\s*\#\s*"
) -> Pattern:
    """
    Create a Regular Expression given a pattern (body) and a prefix
    !Note: This function adds a `#` at the beggining 
    
    :param body: The body of the Regular Expression
    :param expect_module: A flag for marking if the Regular Expression should expect
                          a module name and save it in the groups
    :param comment: A comment describing the use of the regex
    :param prefix: The pattern before the body of the regex
    
    :return: The Regular Expression Pattern 
    """
    groups = ""
    
    if expect_module == NO_MODULE: # NO_MODULE -> #export
        groups = ""
    elif expect_module == MODULE:     
        groups = r"[ \t]+(\S+)"
    elif expect_module == MAYBE_MODULE:
        groups = r"(?:[ \t]+(\S+))?"
    
    return re.compile(rf""" #
            # {comment}: # a brief description of the regex
            ^            # match the beggining of line
            {prefix}     # whitespace before #export
            {body}       # the main body of the regex
            {groups}     # groups of the regex 
            [ \t]*       # spaces or tabs after #export
            $            # end of line
            """, re.MULTILINE | re.VERBOSE)

In [13]:
#export
_re_simple_export =  _make_regex("export[i]?", 
                                NO_MODULE,
    "Match line starting with #export[i] and no module name")

In [14]:
#export
_re_module_export =  _make_regex("export[i]?", 
                                MODULE,
    "Match line starting with #export[i] with a module name")

In [15]:
#export
_re_internal_export = _make_regex("exporti", 
                                 MAYBE_MODULE,
     "Match line starting with #exporti with or without a module name")

In [16]:
#export
_re_export_name = re.compile(r"""
                             ^              # match the beggining of line
                             (?:async\sdef|def|class)
                             \s+            # whitespace
                             ([^\(\s]+)     # catch group that starts with any chatacter except ( or whitespace
                             \s*            # again whitespace
                             (?:\(|:)       # # non-catching group with ( or :
                             """, re.MULTILINE | re.VERBOSE)

In [17]:
#export
_re__all__ = re.compile(r'^__all__\s*=\s*\[([^\]]*)\]', re.MULTILINE)

In [18]:
#export
def search_regex(cell, pattern: str or Pattern) -> Match or None:
    """
    Search a given pattern inside a notebook CODE cell
    
    :param cell: The jupyter notebook cell
    :param pattern: The pattern to be searched
    
    :return: the Match of the pattern inside the cell or None
    """
    
    # only search for 'code' cells
    if cell[CELL_TYPE] != CODE: 
        return None
    
    if isinstance(pattern, str):
        pattern = re.compile(pattern, re.IGNORECASE | re.MULTILINE)
    
    return pattern.search(cell[SOURCE])

In [19]:
#exporti
def _get_module_name(re_match: Match) -> str:
    """
    Get the name of the module for a match 
    of the folowing structure:
    
    #export[i] module_name
    
    :param re_match: The regular expression match
    :return: The name of the module
    """
    return re_match.groups()[0]

In [20]:
#exporti
def _is_internal_export(re_match: Match) -> bool:
    """
    Check if match was of type 
    #exporti 
    meaning internal export (not to be included in __all__)
        
    :param re_match: The regular expression match
    :return: If the match is an internal export
    """
    return _re_internal_export.search(re_match.string) is not None

In [21]:
#export
def is_export_cell(cell, default_module: str) -> Tuple[str, bool] or None:
    """
    Check if a CODE cell should be exported to a python module
    
    :param cell: The jupyter notebook cell
    :param default_module: The default name of the module
                           where the code from the cell should be exported
                           (for #export cells)
    :return: A tuple containing the name of the module where the code
             will be exported and a boolean marking if it is an external
             or internal export
    """
    
    # check if it is a export cell without a module name
    if simple_export := search_regex(cell, _re_simple_export):
        return default_module, _is_internal_export(simple_export)
    
    # check if it is an export with a module name
    if module_export := search_regex(cell, _re_module_export):
        return get_module_name(module_export), _is_internal_export(module_export)
    
    # it wasn't a cell to export
    return None

In [22]:
#exporti
def clean_line(line, clean_export=False) -> str:
    """
    Remove whitespace from a line of code
    
    :param line: The line to be cleaned
    :param clean_export: Flag for removing `#Cell` from the
                         final exported line

    :return: The cleaned line
    """
    if clean_export and line == '\n\n#Cell':
        return '\n\n'
    
    return line.rstrip()

In [23]:
#export
def clean_cell(cell, clean_export=False) -> str:
    """
    Prepare a cell to be exported by:
     - removing the #export
     - adding a `#Cell` marking the code belonged in a cell
     - removing any trailing whitespace
     
    :param cell: The jupyter notebook cell
    :param clean_export: Flag for adding or not adding `#Cell`
    
    :return: The code to be exported
    """
    
    code_lines = cell[SOURCE].split('\n')[1:]
    
    while '#default_export' in code_lines[0] or '#export' in code_lines[0]:
        code_lines.pop(0)
    
    code_lines = ['\n\n#Cell'] + code_lines
        
    return '\n'.join(clean_line(line, clean_export=clean_export) for line in code_lines)

In [24]:
#export
def _add_to_all(module_name: str, names: List[str], line_width: int=100) -> None:
    """
    Append a list of names (classes, functions, variables) to the __all__
    of a given python module (given by name)
    
    :param module_name: The name of the python module where to append the names
    :param names: A list of names to be added to __all__
    :param line_width: The width of the line before adding \n
    """
    
    # check if there are any names to append
    if not names:
        return
    
    # read the current contents of the module
    with open(module_name, 'r', encoding='utf8') as f: 
        text = f.read()
    
    text_wrapper = TextWrapper(width=line_width, 
                               initial_indent='', 
                               subsequent_indent=' ' * len('__all__ = ['), 
                               break_long_words=False)
    
    # find the __all__ inside the python module
    re__all__ = _re__all__.search(text)
    
    # get the indices where the __all__ list starts and end
    start, end = re__all__.start(), re__all__.end()
    
    text__all__ = text_wrapper.wrap(f"{text[start:end-1]}{'' if text[end-2]=='[' else ', '}{', '.join(names)}]")
    
    # add the new names
    with open(module_name, 'w', encoding='utf8') as f:
        f.write(text[:start] + '\n'.join(text__all__) + text[end:])

In [25]:
#export
def is_private(name: str) -> bool:
    """
    Check if a given name is private or not
    
    :param name: The name to be checked
    :return: If the name is private or not
    """
    #Objects with dots are privates if one part is private
    for attribute in name.split('.'):
        if (attribute.startswith('_') and not attribute.startswith('__')) \
            or attribute.startswith('@'): 
            return True
        
    return ('\\' in attribute) or \
           ('^'  in attribute) or \
           ('['  in attribute) or \
           (attribute == 'else')

In [26]:
#export
def get_exported_names(code: str) -> List[str]:
    """
    Get the names to export from a series of line of code
    
    :param code: The code containing functions, classes and/or variables
                 to include in __all__
    :return: A list of names to include in __all__
    """
    names = _re_export_name.findall(code)
    return [f"'{name}'" for name in names if not is_private(name) and not iskeyword(name)]

In [27]:
#exporti
_re_default_module_name = _make_regex('default_export', 
                                     MODULE, 
    "Match a line starting with #default_export with a module name")

In [28]:
#exporti
def _uncapitalize(string: str) -> str:
    """Make the first letter in a string lowercase"""
    return string[0].lower() + string[1:]

In [29]:
#export
def find_default_module(nb_path: Path) -> str:
    """
    Find a line in a Jupyter Notebook cell containing
    #default_export module_name
    !Note: if there is no match then it return the name of the notebook
    
    :param nb_path: The path to the notebook
    :return: The name of the exported python module
    """
    nb = read_notebook(nb_path)
    
    for cell in nb[CELLS]:
        if default_re_match := search_regex(cell, _re_default_module_name):
            return _get_module_name(default_re_match)
        
    return _uncapitalize(nb_path.stem)

In [30]:
#export
def export_notebook(
    nb_path: str or Path,
    default_module_name: str or None=None,
    include_header: bool=True, 
    override: bool=True,
    clean_export: bool=False
) -> None:
    """
    Export a Jupyter Notebook to a python module
    
    :param default_module_name: The name of the exported python module
    :param nb_path: The path to the jupyter notebook
    :param include_header: Flag for adding 'AUTOGENERATED' at the beggining of the module
    :param override: Flag to override the contents of existing python module
                     with the same name as `default_module_name`
    """
    if isinstance(nb_path, str):
        
        if not nb_path.endswith('.ipynb'):
            nb_path += '.ipynb'

        nb_path = Path(nb_path)
    
    if default_module_name is None:
        default_module_name = find_default_module(nb_path)
    
    if not default_module_name.endswith('.py'):
            default_module_name += '.py'
            
    if override:
        if file_exists(default_module_name):
            create_empty_file(default_module_name, nb_path, include_header=include_header)
    
    modules_all = dict()
    nb = read_notebook(nb_path)
    
    # get all the cells that should be exported
    to_export = [is_export_cell(cell, default_module_name) for cell in nb[CELLS]]
    export_cells = [(cell, export) for (cell, export) in zip(nb[CELLS], to_export) if export is not None]
    
    for cell, (module_name, internal_export) in export_cells:
        if not module_name.endswith('.py'):
            module_name += '.py'
        
        code = clean_cell(cell, clean_export=clean_export)
        
        if not file_exists(module_name):
            create_empty_file(module_name, nb_path, include_header=include_header)
        
        with open(module_name, 'a', encoding='utf8') as f:
            f.write(code)
        
        if internal_export:
            continue

        if module_name not in modules_all:
            modules_all[module_name] = []
        else:
            modules_all[module_name] += get_exported_names(code)
    
    for module_name, names in modules_all.items():
        _add_to_all(module_name, names)

    print(f"Exported to {default_module_name}")

In [31]:
#export
def export_notebooks(
    nb_paths: List[Union[str, Path]], 
    default_module_names: Union[List[str], None]=None, 
    include_header: bool=True, 
    override: bool=False,
    clean_export: bool=False
) -> None:
    """
    Export a list of Jupyter Notebooks into python modules
    
    :param nb_paths: A list with paths to Jupyter Notebooks to be exported
    :param default_module_names: A list with names for the python modules
    :param include_header: Flag for adding 'AUTOGENERATED' at the beggining of the module
    :param override: Flag to override the contents of existing python module
                     with the same name as `default_module_name`
    """
    if default_module_names is None:
        default_module_names = [None for _ in nb_paths]
    
    for nb_path, default_module_name in zip(nb_paths, default_module_names):
        export_notebook(nb_path, 
                        default_module_name=default_module_name, 
                        include_header=include_header, 
                        override=override,
                        clean_export=clean_export)

In [32]:
#export
def export_directory(
    path_dir: Union[str, Path],
    include_header: bool=True,
    override: bool=False,
    clean_export: bool=False
) -> None:
    """
    Export all jupyter notebooks from a given directory
    
    :param path_dir: The path to the directory
    :param include_header: Flag for adding 'AUTOGENERATED' at the beggining of the module
    :param override: Flag to override the contents of existing python module
                     with the same name as `default_module_name`
    """
    if isinstance(path_dir, str):
        path_dir = Path(path_dir)
        
    for path in path_dir.iterdir():
        if path.is_file() and path.name.lower().endswith('.ipynb'):
            export_notebook(path,
                            include_header=include_header,
                            override=override,
                            clean_export=clean_export)

In [33]:
#export
def clean_export_notebook(
    path_dir: Union[str, Path],
    override: bool=True
) -> None:
    """
    Export a Jupyter Notebook to a python module
    
    :param nb_path: The path to the jupyter notebook
    :param include_header: Flag for adding 'AUTOGENERATED' at the beggining of the module
    """
    export_notebook(path_dir, override=override, include_header=False, clean_export=True)

In [34]:
export_notebook(nb_path)

Exported to export.py
